<a href="https://colab.research.google.com/github/oenereaty/MP2024/blob/main/MP2024_quiz3(1)_%EC%8A%A4%EB%A7%88%ED%8A%B8%ED%8C%9C%ED%95%99%EA%B3%BC_%EC%B1%84%EA%B2%BD%EC%9B%90(202321639).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **STEP 1**: 데이터

## 데이터 불러오기

In [3]:
pip install datasets evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00


In [4]:
import datasets
import transformers
import evaluate
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [245]:
!unzip archive.zip
df = pd.read_csv("mbti_1.csv")
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

Archive:  archive.zip
replace mbti_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: mbti_1.csv              


In [96]:
df_train['label'] = df_train['type'].apply(lambda x: 0 if 'T' in x else 1)
df_test['label'] = df_test['type'].apply(lambda x: 0 if 'T' in x else 1)

In [97]:
train_dict = {
    'text' : df_train['posts'].tolist(),
    'label' : df_train['label'].tolist()
}

test_dict = {
    'text' : df_test['posts'].tolist(),
    'label' : df_test['label'].tolist()
}

In [98]:
train_data = datasets.Dataset.from_dict(train_dict)
test_data = datasets.Dataset.from_dict(test_dict)
나의데이터 = datasets.dataset_dict.DatasetDict({'train':train_data, 'test':test_data})

## 데이터 전처리1 (tokenizer)

In [99]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [100]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, return_tensor=True)

## 데이터 전처리2 (map)

In [101]:
전처리된나의데이터 = 나의데이터.map(lambda dct: tokenizer(dct["text"], truncation=True))

Map:   0%|          | 0/6940 [00:00<?, ? examples/s]

Map:   0%|          | 0/1735 [00:00<?, ? examples/s]

In [102]:
# def preprocess_function(examples):
#     return tokenizer(
#         examples["text"],
#         truncation=True,
#         padding=True,
#         return_tensors="pt"  # TensorFlow를 사용할 경우에는 "tf"로 변경
#     )

# 전처리된나의데이터 = 나의데이터.map(preprocess_function)

# **STEP 2**: 인공지능 생성

In [105]:
from transformers import AutoModelForSequenceClassification
인공지능 = model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **STEP 3**: 인공지능 학습

### 더 전처리된 데이터. (datacollator)
#### 인공지능은 torch.tensor( ) 를 원한다


In [195]:
from transformers import DataCollatorWithPadding
데이터콜렉터 = datacollator = transformers.DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='pt')

In [196]:
import evaluate
import numpy as np
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### 트레이너 세부 지침 및 트레이너 생성

In [227]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, # 전체문제세트를 2번 공부하라..
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [228]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=전처리된나의데이터["train"],
    eval_dataset=전처리된나의데이터["test"],
    tokenizer=tokenizer,
    data_collator=datacollator,
    compute_metrics=compute_metrics,
)

In [199]:
model.classifier.weight

Parameter containing:
tensor([[-0.0135,  0.0426, -0.0031,  ...,  0.0292, -0.0118, -0.0021],
        [ 0.0134,  0.0318,  0.0042,  ..., -0.0104,  0.0058,  0.0059]],
       device='cuda:0', requires_grad=True)

### tensor 확인

In [211]:
model(**datacollator([tokenizer("This movie was a huge disappointment.")]).to("cuda:0"))

SequenceClassifierOutput(loss=None, logits=tensor([[0.0242, 0.2991]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [242]:
type(전처리된나의데이터['test']['label'][3])

#  label의 타입은 int

int

In [229]:
trainer.predict(전처리된나의데이터['test'])

PredictionOutput(predictions=array([[ 1.4007229, -1.2770596],
       [ 1.2023929, -1.1696299],
       [ 1.6825961, -1.5476327],
       ...,
       [-1.3155096,  1.8836565],
       [ 1.1571057, -1.0538057],
       [ 1.603623 , -1.4794586]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.43619999289512634, 'test_model_preparation_time': 0.0029, 'test_accuracy': 0.8126801152737753, 'test_runtime': 27.4133, 'test_samples_per_second': 63.29, 'test_steps_per_second': 3.976})

In [119]:
trainer.predict(전처리된나의데이터['train'])

PredictionOutput(predictions=array([[0.11977493, 0.00747927],
       [0.119106  , 0.03856627],
       [0.11852184, 0.0451256 ],
       ...,
       [0.14422078, 0.03173084],
       [0.13584644, 0.04645506],
       [0.14232309, 0.01661156]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 1, 0]), metrics={'test_loss': 0.6985427141189575, 'test_model_preparation_time': 0.0033, 'test_accuracy': 0.45878962536023055, 'test_runtime': 116.8942, 'test_samples_per_second': 59.37, 'test_steps_per_second': 3.713})

In [232]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,No log,0.442922,0.002900,0.820749
2,0.344300,0.466555,0.002900,0.815562
3,0.253400,0.567379,0.002900,0.812680


TrainOutput(global_step=1302, training_loss=0.2740645079019432, metrics={'train_runtime': 1195.9368, 'train_samples_per_second': 17.409, 'train_steps_per_second': 1.089, 'total_flos': 2757971240017920.0, 'train_loss': 0.2740645079019432, 'epoch': 3.0})

# **STEP 4**: 예측하기

In [238]:
trainer.predict(전처리된나의데이터['train'])

PredictionOutput(predictions=array([[-1.4476196 ,  1.9587684 ],
       [-0.3915854 ,  0.834722  ],
       [-0.56228054,  0.8323899 ],
       ...,
       [-1.5777289 ,  2.2032845 ],
       [-1.4028325 ,  1.8561907 ],
       [ 1.313267  , -1.2424467 ]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 1, 0]), metrics={'test_loss': 0.25037404894828796, 'test_model_preparation_time': 0.0029, 'test_accuracy': 0.909221902017291, 'test_runtime': 109.3606, 'test_samples_per_second': 63.46, 'test_steps_per_second': 3.969})

In [239]:
trainer.predict(전처리된나의데이터['test'])

PredictionOutput(predictions=array([[ 1.6720916 , -1.5843419 ],
       [ 1.2880664 , -1.3220681 ],
       [ 1.7390537 , -1.6864297 ],
       ...,
       [-1.600014  ,  2.269838  ],
       [ 0.8742746 , -0.81462675],
       [ 1.5992041 , -1.4796865 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.44292154908180237, 'test_model_preparation_time': 0.0029, 'test_accuracy': 0.8207492795389049, 'test_runtime': 28.0962, 'test_samples_per_second': 61.752, 'test_steps_per_second': 3.88})

# **+)** 주어진 text를 분류하기

In [166]:
texts = ["I prefer making decisions based on logic and facts.Analyzing situations objectively is more important to me than considering emotions.",
"When making decisions, I think it’s most important to consider how others feel. Understanding and empathizing with the situation is always my top priority. I feel happiest when relationships are harmonious, and I try to maintain an emotional balance in everything I do."]

In [189]:
text_dict = {
    'text': [texts[0], texts[1]]
    }

text_data = datasets.Dataset.from_dict(text_dict)

In [190]:
tokenized_text = text_data.map(lambda dct: tokenizer(dct["text"], truncation=True))

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [192]:
tokenized_text

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 2
})

In [233]:
model(**datacollator([tokenizer("I prefer making decisions based on logic and facts.Analyzing situations objectively is more important to me than considering emotions.")]).to("cuda:0"))

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.8916, -0.7973]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [236]:
model(**datacollator([tokenizer("When making decisions, I think it’s most important to consider how others feel. Understanding and empathizing with the situation is always my top priority. I feel happiest when relationships are harmonious, and I try to maintain an emotional balance in everything I do.")]).to("cuda:0"))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.9484,  1.5949]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [237]:
logits1 = torch.tensor([ 0.8916, -0.7973])
logits2 = torch.tensor([-0.9484,  1.5949])

확률1 = torch.exp(logits1) / torch.exp(logits1).sum()
확률2 = torch.exp(logits2) / torch.exp(logits2).sum()


확률1, 확률2


(tensor([0.8441, 0.1559]), tensor([0.0729, 0.9271]))